In [1]:
import os
import pandas as pd
from Bio import SeqIO
import numpy as np
import gffutils
from collections import Counter

### Unzip db builds (if necessary):

    tar -zxf nextflow_db.tar.gz
    tar -zxf midasdb_localdb.tar.gz

### Output of nextflow pipeline

In [2]:
my_db_path = '/wynton/home/sirota/clairedubin/MIDAS3_nextflow/testing/nextflow_db'

### Output run generated Chunyu using original MIDAS scripts

Original code here: https://github.com/czbiohub-sf/MIDAS/blob/master/tests/test_db.sh

In [3]:
correct_db_path = '/wynton/home/sirota/clairedubin/MIDAS3_nextflow/testing/midasdb_localdb'

### Comparing folders

Mine is missing genomes.tsv, markers_models, and cleaned_imports because the nextflow pipeline doesn't require these input files to be stored within the database directory. 

In [4]:
!ls {my_db_path}

chunks	gene_annotations  markers  pangenomes  pangenomes_annotation


In [5]:
!ls {correct_db_path}

chunks		 gene_annotations  markers	   pangenomes
cleaned_imports  genomes.tsv	   markers_models  pangenomes_annotation


### Get list of files in each database

In [6]:
db = 'nextflow_db'
my_db_files = []

for path, subdirs, files in os.walk(db):
    for name in files:
        
        if name == 'genomes.tsv' or 'cleaned_imports' in path:
            continue
        
        my_db_files += [os.path.join(path, name).replace(db+'/', '')]

In [7]:
db = 'midasdb_localdb'
correct_db_files = []

for path, subdirs, files in os.walk(db):
    for name in files:
        if '.log' in name: #ignore missing log files
            continue
        if name == 'genomes.tsv' or 'cleaned_imports' in path or 'markers_models' in path:
            continue
        correct_db_files += [os.path.join(path, name).replace(db+'/', '')]

### Compare list of files generated by each database

 

In [8]:
not_matched = []

for f in correct_db_files:
    
    if f in my_db_files:
        continue
            
    file_only = f.split('/')[-1]
    
    if '.log' in f:
        continue
    
    potential_files = [i for i in my_db_files if i.split('/')[-1] == file_only]
    if len(potential_files) > 0:

        
        if '117086' in f:
            potential_files = [i for i in potential_files if '117086' in i]
        elif '117088' in f:
            potential_files = [i for i in potential_files if '117088' in i]
        
        print(f)
        print('possible matches:')
        for i in potential_files:
            print(i)
    
    else:
        print(f)
        print('no matches found')
    
    print()
    
    not_matched+=[f]
        

pangenomes/117088/temp/vsearch/genes.ffn
possible matches:
pangenomes/117088/genes.ffn
pangenomes/117088/temp/cdhit/genes.ffn

pangenomes/117086/temp/vsearch/genes.ffn
possible matches:
pangenomes/117086/genes.ffn
pangenomes/117086/temp/cdhit/genes.ffn



### Unmatched files:
     
     'pangenomes/117086/temp/vsearch/genes.ffn' - can ignore, this is the same as nextflow_db/pangenomes/117086/genes.ffn, just a different path

In [9]:
not_matched

['pangenomes/117088/temp/vsearch/genes.ffn',
 'pangenomes/117086/temp/vsearch/genes.ffn']

In [10]:
#genes.ffn files are the same
!diff <(sort midasdb_localdb/pangenomes/117086/temp/vsearch/genes.ffn) <(sort nextflow_db/pangenomes/117086/genes.ffn)
!diff <(sort midasdb_localdb/pangenomes/117088/temp/vsearch/genes.ffn) <(sort nextflow_db/pangenomes/117088/genes.ffn)


### Are the contents of each file the same?

In [11]:
#ignore lines with these words
diff_exclude = '" -I "'.join(['-I "clairedubin','pollard', #names in most paths
                        '^\#.*', #exclude comment lines  
                        'time', 'tm_sec', 'tm_min', 'tm_hour', 'tm_mday',
                        'tm_mon', 'tm_wday', 'tm_yday', 'Date', #timestamp stuff
                        'Database name', #path to databases (some aren't full paths)
                        '\<Statistics_',  #comments in resfinder outputs
                             
                             ])
diff_exclude += '"'

In [12]:
#these will cause diff to fail
binary_files = ['markers/phyeco/phyeco.fa.bwt',
               'markers/phyeco/phyeco.fa.sa']

In [13]:
different_contents = {}
same_contents = []
different_but_ok_files = []

for f in correct_db_files:
        
    if f in not_matched or f in binary_files:
        continue
        
    if 'temp' in f or 'tmp' in f:#skip intermediate files
        continue
        
    #check if contents are the same, excluding words specified above and sorting lines
    cmd = f"diff {diff_exclude} <(sort nextflow_db/{f}) <(sort midasdb_localdb/{f})"
    diff_out = !{cmd}
    
    if len(diff_out) == 0:
        same_contents += [f]
        continue
    
    different_contents[f] =  [f'nextflow_db/{f}', f'midasdb_localdb/{f}', diff_out]
        

In [14]:
len(same_contents), len(different_contents)

(342, 38)

### Which subfolders have differing files?

In [15]:
subs = []

for i in different_contents.keys():
    
    subfolder = i.split('/')[0]
    subs += [subfolder]
    
Counter(subs)


Counter({'pangenomes_annotation': 33, 'gene_annotations': 4, 'markers': 1})

### Compare differing files in gene_annotations directory

These are the same - diff just can't handle the DB files

In [16]:
g = []
for i in different_contents.keys():
    if 'gene_annotations/' in i:
        print(i)
        g += [i]

gene_annotations/117086/GCF_900752885.1/GCF_900752885.1.gff.db
gene_annotations/117086/GCA_900552055.1/GCA_900552055.1.gff.db
gene_annotations/117088/GCA_007120565.1/GCA_007120565.1.gff.db
gene_annotations/117088/GCA_007135585.1/GCA_007135585.1.gff.db


In [17]:
for i in g:

    test_gff = gffutils.interface.FeatureDB(f'nextflow_db/{i}')
    correct_gff = gffutils.interface.FeatureDB(f'midasdb_localdb/{i}')

    c = correct_gff.all_features()

    for i in test_gff.all_features():
        if i != next(c):
            print(i)

In [18]:
ignore_diffs = []
ignore_diffs += [i for i in different_contents.keys() if 'gene_annotations/' in i]

### Compare differing files in markers directory

Header files are just different because of the date/path in the header.

In [19]:
for i in different_contents.keys():
    if 'markers/' in i:
        print(i)

markers/phyeco/phyeco.fa.header


In [20]:
f = 'markers/phyeco/phyeco.fa'
cmd = f"diff {diff_exclude} <(sort nextflow_db/{f}) <(sort midasdb_localdb/{f})"
!{cmd}

In [21]:
f = 'markers/phyeco/phyeco.fa.header'
cmd = f"diff {diff_exclude} <(sort nextflow_db/{f}) <(sort midasdb_localdb/{f})"
!{cmd}

92,99c92,99
< 	tm_hour=17
< 	tm_mday=20
< 	tm_min=55
< 	tm_mon=1
< 	tm_sec=55
< 	tm_wday=5
< 	tm_yday=50
< 	tm_year=126
---
> 	tm_hour=14
> 	tm_mday=24
> 	tm_min=26
> 	tm_mon=8
> 	tm_sec=46
> 	tm_wday=2
> 	tm_yday=267
> 	tm_year=124


In [22]:
ignore_diffs += ['markers/phyeco/phyeco.fa.header']

## Compare differing files in pangenomes_annotation directory


In [23]:
for i in different_contents.keys():
    if 'pangenomes_annotation/' in i:
        print(i)

pangenomes_annotation/01_mge/117086/GCA_900552055.1/resfinder_output/pheno_table.txt
pangenomes_annotation/01_mge/117086/GCA_900552055.1/genomad_output/GCA_900552055.1_score_calibration/GCA_900552055.1_calibrated_aggregated_classification.npz
pangenomes_annotation/01_mge/117086/GCA_900552055.1/genomad_output/GCA_900552055.1_score_calibration/GCA_900552055.1_calibrated_nn_classification.npz
pangenomes_annotation/01_mge/117086/GCA_900552055.1/genomad_output/GCA_900552055.1_score_calibration/GCA_900552055.1_provirus_calibrated_nn_classification.npz
pangenomes_annotation/01_mge/117086/GCA_900552055.1/genomad_output/GCA_900552055.1_score_calibration/GCA_900552055.1_provirus_calibrated_aggregated_classification.npz
pangenomes_annotation/01_mge/117086/GCA_900552055.1/genomad_output/GCA_900552055.1_nn_classification/GCA_900552055.1_nn_classification.npz
pangenomes_annotation/01_mge/117086/GCA_900552055.1/genomad_output/GCA_900552055.1_nn_classification/GCA_900552055.1_nn_classification.tsv
pan

### Genomad outputs

TSVs and .npz files are the same, except for a <0.001 rounding difference in a few scores. 



In [24]:
genomad_tsvs = [i for i in different_contents.keys() if 'genomad_output' in i and i.endswith('.tsv')]

In [25]:
for f in genomad_tsvs:
    
    df1 = pd.read_csv(f'nextflow_db/{f}', sep='\t', index_col=0)
    df2 = pd.read_csv(f'midasdb_localdb/{f}', sep='\t', index_col=0)

    numeric_cols = df1.select_dtypes(include=[np.number]).columns
    str_cols = df1.select_dtypes(exclude=[np.number]).columns

    for gene, row1 in df1.iterrows():
        row2 = df2.loc[gene]
        
        if row1.equals(row2):
            continue
            

        if not row1[str_cols].equals(row2[str_cols]):
            print(gene)
            print("String/Categorical mismatch:")
            print("Row 1:", row1)
            print("Row 2:", row2)
            print()
            continue


        numeric_diff = abs(row1[numeric_cols].fillna(0) - row2[numeric_cols].fillna(0)).sum()
        
        if numeric_diff <= 0.001:
            continue
        else:
            print(gene)
            print("Numeric mismatch:")
            print("Row 1:", row1)
            print("Row 2:", row2)
            print(f"Total numeric difference: {numeric_diff}")
            print()

In [26]:
npz_files = []

for f, (test_path, correct_path, diff_out) in different_contents.items():

    if '.npz' in f and 'genomad' in f:
        npz_files += [(test_path, correct_path)]
        continue

for (f1_path, f2_path) in npz_files:
    
    f1 = np.load(f1_path)
    f2 = np.load(f2_path)
    for i in f1:

        if (f1[i] == f2[i]).all():
            continue

        else:
            diffs = np.where(abs(f1[i] - f2[i]) > 0.001)
            if len(diffs) != 0:
                print(diffs)


(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int

In [27]:
[i for i in different_contents.keys() if 'pangenomes_annotation/' in i and 'genomad' in i and not ('.npz' in i or '.tsv' in i)]

[]

In [28]:
ignore_diffs += [i for i in different_contents.keys() if 'pangenomes_annotation/' in i and 'genomad' in i and ('.npz' in i or '.tsv' in i)]

In [29]:
[i for i in different_contents.keys() if i not in ignore_diffs]

['pangenomes_annotation/01_mge/117086/GCA_900552055.1/resfinder_output/pheno_table.txt',
 'pangenomes_annotation/01_mge/117086/GCF_900752885.1/resfinder_output/pheno_table.txt',
 'pangenomes_annotation/01_mge/117088/GCA_007135585.1/resfinder_output/pheno_table.txt',
 'pangenomes_annotation/01_mge/117088/GCA_007120565.1/resfinder_output/pheno_table.txt']

In [33]:
!diff 'nextflow_db/pangenomes_annotation/01_mge/117086/GCA_900552055.1/resfinder_output/pheno_table.txt' 'midasdb_localdb/pangenomes_annotation/01_mge/117086/GCA_900552055.1/resfinder_output/pheno_table.txt'

68a69
> cephalotin	beta-lactam	No resistance	0	


In [48]:
!diff 'nextflow_db/pangenomes_annotation/01_mge/117086/GCF_900752885.1/resfinder_output/pheno_table.txt' 'midasdb_localdb/pangenomes_annotation/01_mge/117086/GCF_900752885.1/resfinder_output/pheno_table.txt'

68a69
> cephalotin	beta-lactam	No resistance	0	


In [32]:
!cat 'midasdb_localdb/pangenomes_annotation/01_mge/117086/GCA_900552055.1/resfinder_output/pheno_table.txt'

# ResFinder phenotype results.
# Sample: GCA_900552055.1.fna
# 
# The phenotype 'No resistance' should be interpreted with
# caution, as it only means that nothing in the used
# database indicate resistance, but resistance could exist
# from 'unknown' or not yet implemented sources.
# 
# The 'Match' column stores one of the integers 0, 1, 2, 3.
#      0: No match found
#      1: Match < 100% ID AND match length < ref length
#      2: Match = 100% ID AND match length < ref length
#      3: Match = 100% ID AND match length = ref length
# If several hits causing the same resistance are found,
# the highest number will be stored in the 'Match' column.

# Antimicrobial	Class	WGS-predicted phenotype	Match	Genetic background
gentamicin	aminoglycoside	No resistance	0	
tobramycin	aminoglycoside	No resistance	0	
streptomycin	aminoglycoside	No resistance	0	
amikacin	aminoglycoside	No resistance	0	
isepamicin	aminoglycoside	No resistance	0	
dibekacin	aminoglycoside	No resistance	0	
kanamycin	amino

In [46]:
! midasdb_localdb/pangenomes_annotation/01_mge/117086/GCA_900552055.1/resfinder_output/

disinfinder_blast		     pheno_table.txt
DisinFinder_Hit_in_genome_seq.fsa    resfinder_blast
DisinFinder_Resistance_gene_seq.fsa  ResFinder_Hit_in_genome_seq.fsa
DisinFinder_results_table.txt	     ResFinder_Resistance_gene_seq.fsa
DisinFinder_results_tab.txt	     ResFinder_results_table.txt
DisinFinder_results.txt		     ResFinder_results_tab.txt
GCA_900552055.json		     ResFinder_results.txt


In [47]:
!cat midasdb_localdb/pangenomes_annotation/01_mge/117086/GCA_900552055.1/resfinder_output/*txt

Disinfectants
No hit found

Resistance gene	Identity	Alignment Length/Gene Length	Coverage	Position in reference	Contig	Position in contig	Phenotype	Accession no.
# ResFinder phenotype results.
# Sample: GCA_900552055.1.fna
# 
# The phenotype 'No resistance' should be interpreted with
# caution, as it only means that nothing in the used
# database indicate resistance, but resistance could exist
# from 'unknown' or not yet implemented sources.
# 
# The 'Match' column stores one of the integers 0, 1, 2, 3.
#      0: No match found
#      1: Match < 100% ID AND match length < ref length
#      2: Match = 100% ID AND match length < ref length
#      3: Match = 100% ID AND match length = ref length
# If several hits causing the same resistance are found,
# the highest number will be stored in the 'Match' column.

# Antimicrobial	Class	WGS-predicted phenotype	Match	Genetic background
gentamicin	aminoglycoside	No resistance	0	
tobramycin	aminoglycoside	No resistance	0	
streptomycin	aminoglyco